<a href="https://colab.research.google.com/github/KC-ai/APPM4600/blob/main/NumericsHW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
from numpy.linalg import norm, solve

Problem 1

In [15]:
def F1(xy):
    x, y = xy
    return np.array([x**2 + y**2 - 4,
                     np.exp(x) + y - 1], dtype=float)

def jacobian(xy):
    x, y = xy
    return np.array([[2*x,    2*y],
                     [np.exp(x), 1   ]], dtype=float)


def broyden(x0, tol=1e-8, max_iter=50):
    x = np.array(x0, dtype=float)
    B = np.eye(2)
    Fx = F1(x)
    for k in range(max_iter):
        if norm(Fx) < tol:
            return x, k

        s = solve(B, -Fx)
        x_new = x + s
        Fx_new = F1(x_new)

        y = Fx_new - Fx
        denom = np.dot(s, s)
        if abs(denom) < 1e-14:
            return x_new, k+1

        B += np.outer(y - B @ s, s) / denom

        x, Fx = x_new, Fx_new

    return x, max_iter


def lazy_newton(x0, tol=1e-8, max_iter=50):
    x = np.array(x0, dtype=float)

    for k in range(max_iter):
        Fx = F1(x)
        if norm(Fx) < tol:
            return x, k

        J0 = jacobian(x)

        if np.linalg.det(J0) == 0:
            print("Jacobian is singular. Try a different initial guess.")
            return None, None

        x = x + solve(J0, -Fx)
    return x, max_iter

In [16]:

initial_guesses = [(1,1), (1,-1), (0,0)]

for g in initial_guesses:
    print(f"Initial guess: {g}")

    sol_b, it_b = broyden(g)
    print(f"Broyden-- Iterations:{it_b} --Solution:  {sol_b}")

    sol_ln, it_ln = lazy_newton(g)
    print(f"Lazy Newton -- Iterations:{it_ln} --Solution:  {sol_ln}")


Initial guess: (1, 1)
Broyden-- Iterations:20 --Solution:  [ 1.00416874 -1.72963729]
Lazy Newton -- Iterations:7 --Solution:  [-1.81626407  0.8373678 ]
Initial guess: (1, -1)
Broyden-- Iterations:12 --Solution:  [ 1.00416874 -1.72963729]
Lazy Newton -- Iterations:4 --Solution:  [ 1.00416874 -1.72963729]
Initial guess: (0, 0)
Broyden-- Iterations:19 --Solution:  [-1.81626407  0.8373678 ]
Jacobian is singular. Try a different initial guess.
Lazy Newton -- Iterations:None --Solution:  None


Problem 2

In [17]:
def F(x0):
    x, y, z = x0
    system = np.array([
        x + np.cos(x*y*z) - 1,
        (1 - x)**0.25 + y + 0.05*z**2 - 0.15*z - 1,
        -x**2 - 0.1*y**2 + 0.01*y + z - 1
    ])
    return system

def Jacobian(X):
    x, y, z = X

    dF1dx1 = 1.0 - np.sin(x*y*z)*(y*z)
    dF1dx2 =       - np.sin(x*y*z)*(x*z)
    dF1dx3 =       - np.sin(x*y*z)*(x*y)

    dF2dx1 = -1.0 / (4.0*(1.0 - x)**0.75)
    dF2dx2 = 1.0
    dF2dx3 = 0.10*z - 0.15

    dF3dx1 = -2.0*x
    dF3dx2 = -0.2*y + 0.01
    dF3dx3 = 1.0

    jacob = np.array([
        [dF1dx1, dF1dx2, dF1dx3],
        [dF2dx1, dF2dx2, dF2dx3],
        [dF3dx1, dF3dx2, dF3dx3]
    ])

    return jacob

In [18]:
def newton(x0, tol=1e-6, max_iter=50):
    x = x0.copy()
    for k in range(max_iter):
        Fx = F(x)
        if np.linalg.norm(Fx) < tol:
            return x, k+1

        delta = np.linalg.solve(Jacobian(x), -Fx)
        x += delta
    return x, max_iter

In [20]:
def steepest_descent(x0, tol=1e-6, max_iter=2000, alpha=1e-3):
    x = x0.copy()
    for k in range(max_iter):
        Fx = F(x)
        if np.linalg.norm(Fx) < tol:
            return x, k+1
        grad_phi = Jacobian(x).T @ Fx
        x -= alpha * grad_phi
    return x, max_iter

In [21]:
def hybrid_method(x0, tol_sd=5e-2, tol_newt=1e-6):
    x = x0.copy()
    alpha = 1e-3
    for _ in range(2000):
        Fx = F(x)
        if np.linalg.norm(Fx) < tol_sd:
            break
        grad_phi = Jacobian(x).T @ Fx
        x -= alpha * grad_phi
    return newton(x, tol=tol_newt, max_iter=50)

In [22]:
x0 = np.array([0.75, 0.75, 0.75])

# Newton
sol_n, it_n = newton(x0)
print(f"Newton's Method: Number of Iterations {it_n} with Solution {sol_n}")
# Steepest Descent
sol_s, it_s = steepest_descent(x0)
print(f"Steepest Descent Method: Number of Iterations {it_s} with Solution {sol_s}")
# Steepest Descent -> Newton
sol_h, it_h = hybrid_method(x0)
print(f"Hybrid Method: Number of Iterations {it_h} with Solution {sol_h}")

<ipython-input-17-cc68a09e8dd0>:5: RuntimeWarning: invalid value encountered in scalar power
  (1 - x)**0.25 + y + 0.05*z**2 - 0.15*z - 1,
<ipython-input-17-cc68a09e8dd0>:17: RuntimeWarning: invalid value encountered in scalar power
  dF2dx1 = -1.0 / (4.0*(1.0 - x)**0.75)


Newton's Method: Number of Iterations 50 with Solution [nan nan nan]
Steepest Descent Method: Number of Iterations 2000 with Solution [0.13956387 0.25629221 1.05511546]
Hybrid Method: Number of Iterations 4 with Solution [1.29606844e-17 1.00000000e-01 1.00000000e+00]
